In [1]:
import pandas as pd

### Listen Events Table Old

In [3]:
df = pd.read_csv(
    "/home/lupusruber/music_analytics/data/data/events.csv", encoding="utf-8"
)

In [4]:
transform_string = (
    lambda x: "".join([("_" + i if i.isupper() else i) for i in x]).strip().lower()
)

In [5]:
df.columns = list(map(transform_string, df.columns.tolist()))
df = df.rename({"ts": "timestamp"}, axis=1)

In [6]:
df.head(1)

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [7]:
df.query("timestamp == 1616086802000")

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,0,Ayo,Welcome Into My World,264.56772,1616086802000,9782,Logged In,free,2,Flagstaff,86004,AZ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-111.373463,35.380939,9783,Parker,Haley,F,1616086800000


In [8]:
print(
    df[["session_id", "user_id"]].drop_duplicates().shape,
    df[["session_id", "user_id"]].shape,
)

(14662, 2) (100000, 2)


In [9]:
new_df = df[["session_id", "user_id"]].groupby("session_id").aggregate(lambda x: x)

In [10]:
import numpy as np

In [11]:
new_df["user_id"] = new_df["user_id"].apply(
    lambda x: list(x) if isinstance(x, (list, np.ndarray)) else [x]
)

In [12]:
new_df["truth"] = new_df["user_id"].apply(lambda x: len(set(x)) == 1)

In [13]:
new_df["truth"].unique()

array([ True])

In [14]:
type(new_df["user_id"].values[0])

list

In [15]:
df.groupby("session_id").count().reset_index()

,session_id,_unnamed: 0,artist,song,duration,timestamp,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
0,9,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,10,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,12,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13,13
3,28,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
4,31,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,55788,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14658,55805,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14,14
14659,55848,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3
14660,55882,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [16]:
gdf = df.groupby("gender")["gender"].count() / len(df)

In [17]:
df.query("user_id==748 and item_in_session==30").sort_values(
    "timestamp", ascending=True
).head(10)

,_unnamed: 0,artist,song,duration,timestamp,session_id,auth,level,item_in_session,city,zip,state,user_agent,lon,lat,user_id,last_name,first_name,gender,registration
650,650,Irma Thomas,The Same Love That Made Me Laugh [LP Version],195.36934,1616087608000,747,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000
80143,80143,Chris Vickery,Jokers and Thieves,191.71220,1616302962000,49237,Logged In,paid,30,Egg Harbor Township,8234,NJ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-74.620728,39.388393,748,Ramirez,Kyra,F,1616086800000


In [18]:
result = (
    df.groupby("session_id")
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# result['count'].sum() == n_rows
result

,session_id,count
7220,37432,219
7958,38794,216
13931,51602,199
4659,30891,197
8117,39079,178
...,...,...
4605,30649,1
4637,30787,1
4635,30784,1
4634,30781,1


In [19]:
df.query("session_id == 37432")[
    ["artist", "song", "duration", "item_in_session", "user_id"]
]["user_id"].unique()

array([17692])

In [20]:
session_and_user = (
    df.groupby(["session_id", "user_id"])
    .size()
    .reset_index(name="count")
    .sort_values(by="count", ascending=False)
)
# df.groupby(['sessionId', 'userId']).size().reset_index(name='count')['count].sum() == n_rows
session_and_user

,session_id,user_id,count
7220,37432,17692,219
7958,38794,17940,216
13931,51602,19676,199
4659,30891,16304,197
8117,39079,17780,178
...,...,...,...
4605,30649,1903,1
4637,30787,10466,1
4635,30784,13727,1
4634,30781,17442,1


In [21]:
session_and_user.groupby("user_id").size().reset_index(name="count").sort_values(
    "count", ascending=False
)

,user_id,count
5726,13473,17
3371,8010,16
121,288,15
3119,7357,13
3783,8943,12
...,...,...
8439,19986,1
8425,19958,1
8426,19959,1
8428,19964,1


In [22]:
session_and_user.sort_values("user_id")

,session_id,user_id,count
4132,28898,3,3
10935,44377,8,1
6093,35076,8,3
0,9,10,1
10626,43735,11,5
...,...,...,...
14085,52141,19996,1
13523,50407,19997,5
2870,19996,19997,1
7136,37261,20001,37


In [23]:
df["user_id"].unique().shape[0] == df["session_id"].unique().shape[0]

False

In [24]:
df.query("user_id == 19997").groupby("session_id").aggregate(lambda x: x)[
    "item_in_session"
]

session_id
19996                  0
50407    [0, 1, 3, 5, 6]
Name: item_in_session, dtype: object

### Auth Events Table

In [25]:
import pandas as pd

auth_events = pd.read_csv("/home/lupusruber/music_analytics/data/auth_events.csv")
print(auth_events.head(1)["userAgent"].values[0])
auth_events["ts"] = pd.to_datetime(auth_events["ts"], unit="ms")
auth_events["registration"] = pd.to_datetime(auth_events["registration"], unit="ms")

Mozilla/5.0 (X11; Linux x86_64; rv:31.0) Gecko/20100101 Firefox/31.0


In [26]:
mask = auth_events.query("userId == 75")[["ts"]].idxmin()
auth_events.loc[mask]

,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
60,2021-01-23 12:42:01,814,free,5,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,True


In [27]:
auth_events.groupby("userId").aggregate(
    {
        "ts": ["min", "max"],
    }
).reset_index()

userId                  ts                    
                          min                 max
0     2.0 2021-01-06 18:10:32 2021-01-06 18:10:32
1     3.0 2021-04-01 11:41:41 2021-10-27 17:51:22
2     4.0 2021-06-25 13:16:20 2021-07-16 11:47:04
3     5.0 2021-01-13 18:03:17 2021-10-20 16:35:58
4     6.0 2021-05-10 04:31:22 2021-11-27 12:25:43
..    ...                 ...                 ...
84   97.0 2021-03-17 03:44:39 2021-11-12 15:51:50
85   98.0 2021-01-14 18:41:11 2021-07-23 15:16:10
86   99.0 2021-01-31 09:47:32 2021-11-26 12:02:24
87  100.0 2021-11-14 11:26:18 2021-11-14 11:26:18
88  101.0 2021-07-13 18:27:42 2021-07-13 18:27:42

[89 rows x 3 columns]

### Page View Events Table

In [28]:
page_view = pd.read_csv("/home/lupusruber/music_analytics/data/page_view_events.csv")
page_view.drop(["userAgent"], inplace=True, axis=1)
page_view["ts"] = pd.to_datetime(page_view["ts"], unit="ms")
page_view["registration"] = pd.to_datetime(page_view["registration"], unit="ms")
page_view.sample(3)[["ts", "registration"]]

,ts,registration
123237,2021-11-20 15:50:19,2021-01-01
4660,2021-01-20 03:07:29,2021-01-01
37514,2021-05-06 18:38:08,2021-01-01


In [29]:
page_view["registration"].astype(str).str.len().min()

np.int64(3)

In [30]:
user_and_first_ts = (
    page_view.groupby("userId")
    .aggregate({"ts": "min", "registration": "min"})
    .reset_index()
)
user_and_first_ts.query("userId==25")

,userId,ts,registration
23,25.0,2021-01-05 05:01:35,2021-01-01


In [31]:
(
    page_view.merge(user_and_first_ts, on=["ts", "userId"])
    .query("itemInSession == 0")[["ts", "userId"]]
    .reset_index()
    .sort_values(by="userId")
    .groupby("userId")
    .size()
    .reset_index(name="count")
    .query("count!=1")
)

,userId,count


In [32]:
page_view["page"].unique().tolist()

['NextSong',
 'Home',
 'Upgrade',
 'Logout',
 'Login',
 'Help',
 'Downgrade',
 'Settings',
 'Save Settings',
 'About',
 'Error',
 'Submit Upgrade',
 'Submit Downgrade',
 'Cancel',
 'Cancellation Confirmation']

In [33]:
page_view["method"].unique().tolist()

['PUT', 'GET']

In [34]:
page_view.columns.tolist()

['ts',
 'sessionId',
 'page',
 'auth',
 'method',
 'status',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration',
 'artist',
 'song',
 'duration']

In [35]:
counts = page_view.groupby("userId").size().reset_index(name="count")
max_mask = counts["count"].idxmax()
counts.loc[max_mask]

userId       75.0
count     13149.0
Name: 73, dtype: float64

In [36]:
mask = page_view.query("userId == 75")[["ts"]].idxmin()
page_view.loc[mask]

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
301,2021-01-02 07:37:40,74,NextSong,Logged In,PUT,200,free,0,Saint George,84770,...,-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,Hot Live Guys,I'm A Killer,138.13506


In [85]:
sessions_with_no_users = pd.DataFrame(
    page_view[["sessionId", "userId"]][page_view["userId"].isna()][
        "sessionId"
    ].reset_index(drop=True)
)
sessions_with_no_users.merge(page_view, on="sessionId")[
    ["sessionId", "userId"]
].groupby(["sessionId", "userId"]).size().reset_index(name="count").query(
    "count!=1"
).sort_values(
    "userId"
)

,sessionId,userId,count
0,1,2.0,8
1141,6527,3.0,34
970,5678,3.0,50
751,4653,3.0,2
325,2541,3.0,32
...,...,...,...
1660,8740,99.0,52
1636,8634,99.0,315
1634,8622,99.0,205
1609,8483,100.0,16


In [88]:
page_view["sessionId"].hasnans

False

### Status Change Events Table

In [38]:
import pandas as pd

status_change = pd.read_csv(
    "/home/lupusruber/music_analytics/data/status_change_events.csv"
)

status_change.rename({"ts": "timestamp"}, axis=1, inplace=True)

status_change["timestamp"] = pd.to_datetime(status_change["timestamp"], unit="ms")
status_change["registration"] = pd.to_datetime(status_change["registration"], unit="ms")

status_change["registration"].max()

Timestamp('2021-01-01 00:00:00')

In [39]:
status_change.query("userId==25")

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration


In [40]:
status_change["registration"].astype(str).str.len().min(), status_change[
    "registration"
].astype(str).str.len().max()

(np.int64(10), np.int64(10))

In [41]:
pd.to_datetime(1609459200000, unit="ms")

Timestamp('2021-01-01 00:00:00')

In [42]:
status_change.groupby("userId").aggregate({"timestamp": "min", "registration": "min"})

,timestamp,registration
userId,,
3,2021-03-23 15:06:09,2021-01-01
10,2021-01-08 17:03:07,2021-01-01
13,2021-06-03 07:09:13,2021-01-01
20,2021-03-16 09:25:21,2021-01-01
21,2021-09-05 16:14:19,2021-01-01
26,2021-08-11 16:18:13,2021-01-01
27,2021-01-27 07:13:06,2021-01-01
28,2021-08-06 16:32:14,2021-01-01
30,2021-01-24 11:09:03,2021-01-01


In [43]:
status_change["auth"].unique().tolist()

['Logged In']

In [44]:
status_change.columns.tolist()

['timestamp',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'userAgent',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

In [45]:
mask = status_change.query("userId == 75")[["timestamp"]].idxmin()
status_change.loc[mask]

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
8,2021-01-29 05:33:37,1008,Logged In,free,3,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


### Listen Events Table

In [2]:
import pandas as pd

listen_events = pd.read_csv(
    r"/home/lupusruber/music_analytics/data/listen_events.csv", encoding="utf-8"
)
listen_events.rename({"ts": "timestamp"}, axis=1, inplace=True)
listen_events["registration"] = pd.to_datetime(listen_events["registration"], unit="ms")
listen_events["timestamp"] = pd.to_datetime(listen_events["timestamp"], unit="ms")

In [4]:
listen_events.query('artist=="M"')

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
696,M,Close To Me,238.41914,2021-01-04 23:58:18,241,Logged In,paid,101,Himrod,14842,NY,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,-76.981134,42.600505,55,Williams,Celeste,F,2021-01-01
3057,M,Close To Me,238.41914,2021-01-15 04:17:52,572,Logged In,paid,96,Teaneck,7666,NJ,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",-74.011478,40.890317,69,Hill,Christopher,M,2021-01-01
7120,M,Le Bug (Live 2005),259.83954,2021-02-03 14:38:50,969,Logged In,free,8,Lawton,73505,OK,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",-98.481242,34.576274,41,Wade,Ashton,M,2021-01-01
8214,M,Le Bug (Live 2005),259.83954,2021-02-10 01:40:22,836,Logged In,paid,15,Palm Harbor,34683,FL,"""Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537....",-82.761134,28.087706,98,Weeks,Constantino,M,2021-01-01
9887,M,Close To Me,238.41914,2021-02-17 08:07:31,1489,Logged In,paid,93,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01
13429,M,Close To Me,238.41914,2021-03-03 08:14:51,1828,Logged In,paid,51,Houston,77057,TX,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:24.0) G...,-95.487974,29.744081,36,Jackson,Alexander,M,2021-01-01
13546,M,Le Bug (Live 2005),259.83954,2021-03-03 16:47:33,1831,Logged In,paid,21,Himrod,14842,NY,Mozilla/5.0 (compatible; MSIE 10.0; Windows NT...,-76.981134,42.600505,55,Williams,Celeste,F,2021-01-01
20296,M,Je Suis Une Cigarette,215.95383,2021-03-31 12:09:26,2569,Logged In,paid,19,Oklahoma City,73120,OK,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-97.573674,35.581316,50,Rowe,Levi,M,2021-01-01
25638,M,Le Bug (Live 2005),259.83954,2021-04-16 15:22:48,3009,Logged In,paid,6,Charter Oak,51439,IA,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,-95.590386,42.088789,62,Atkins,Lily,F,2021-01-01
27643,M,Je Suis Une Cigarette,215.95383,2021-04-22 16:01:55,3061,Logged In,paid,44,Oklahoma City,73120,OK,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-97.573674,35.581316,50,Rowe,Levi,M,2021-01-01


In [47]:
pd.to_datetime(listen_events["registration"], origin="unix", unit="ms").min()

Timestamp('2021-01-01 00:00:00')

In [48]:
listen_events.query("userId==75").head(3)

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01
319,A-Mei,Intro,71.88853,2021-01-02 20:52:55,212,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01
321,Amorphis,And I Hear You Call,280.97260,2021-01-02 20:54:06,212,Logged In,free,1,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [49]:
listen_events.columns.tolist()

['artist',
 'song',
 'duration',
 'timestamp',
 'sessionId',
 'auth',
 'level',
 'itemInSession',
 'city',
 'zip',
 'state',
 'userAgent',
 'lon',
 'lat',
 'userId',
 'lastName',
 'firstName',
 'gender',
 'registration']

In [50]:
# assert listen_events.query("timestamp <= registration").shape[0] == 0, "Invalid Rows"

In [51]:
listen_events["auth"].unique().tolist()

['Logged In']

In [52]:
mask = listen_events.query("userId == 75")[["timestamp"]].idxmin()
listen_events.loc[mask]

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [53]:
!ls -l

total 108
-rw-r--r-- 1 lupusruber lupusruber   1308 Sep 19 11:38 spark.ipynb
-rw-r--r-- 1 lupusruber lupusruber 104561 Sep 23 12:16 tests.ipynb


### Other

In [54]:
mask = listen_events.query("userId == 75")[["timestamp"]].idxmin()
listen_events.loc[mask]

,artist,song,duration,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
276,Hot Live Guys,I'm A Killer,138.13506,2021-01-02 07:37:40,74,Logged In,free,0,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [55]:
mask = status_change.query("userId == 75")[["timestamp"]].idxmin()
status_change.loc[mask]

,timestamp,sessionId,auth,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration
8,2021-01-29 05:33:37,1008,Logged In,free,3,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75,Buck,Oscar,M,2021-01-01


In [56]:
mask = page_view.query("userId == 75")[["ts"]].idxmin()
page_view.loc[mask]

,ts,sessionId,page,auth,method,status,level,itemInSession,city,zip,...,lon,lat,userId,lastName,firstName,gender,registration,artist,song,duration
301,2021-01-02 07:37:40,74,NextSong,Logged In,PUT,200,free,0,Saint George,84770,...,-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,Hot Live Guys,I'm A Killer,138.13506


In [57]:
mask = auth_events.query("userId == 75")[["ts"]].idxmin()
auth_events.loc[mask]

,ts,sessionId,level,itemInSession,city,zip,state,userAgent,lon,lat,userId,lastName,firstName,gender,registration,success
60,2021-01-23 12:42:01,814,free,5,Saint George,84770,UT,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-113.609837,37.185679,75.0,Buck,Oscar,M,2021-01-01,True


In [58]:
print(auth_events["userId"].unique().shape[0])
print(page_view["userId"].unique().shape[0])
print(listen_events["userId"].unique().shape[0])
print(status_change["userId"].unique().shape[0])

90
101
100
35


In [59]:
set(page_view["userId"].unique()).difference(set(listen_events["userId"].unique()))

{np.float64(nan)}

In [60]:
page_view.rename({"ts": "timestamp"}, axis=1, inplace=True)

In [61]:
new_df = page_view.merge(
    listen_events, on=["userId", "sessionId", "itemInSession"], validate="one_to_one"
)

In [62]:
set(new_df.columns[~new_df.columns.str.contains("x")].tolist()).intersection(
    new_df.columns[~new_df.columns.str.contains("y")].tolist()
)

{'itemInSession',
 'method',
 'page',
 'sessionId',
 'status',
 'userAgent',
 'userId'}

In [63]:
assert (new_df["timestamp_x"] == new_df["timestamp_y"]).unique()[0]

In [64]:
hash(55555555555) * hash("1") / hash("1")

55555555555.0